In [1]:
import ROOT
from fitter import FitHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x837b210


In [2]:
default_lumi = 2000
default_hel = {
    "lumi_frac": 1.,
    "e_pol": 0.,
    "p_pol": 0.,
}

# https://arxiv.org/pdf/1506.07830
# However, https://arxiv.org/pdf/2503.19983 cites the above but assigns 0.45 to both mixed pols
# LCVision scenario uses 3 ab_inv instead of 2
ilc_250_h20_lumi = 2000
ilc_250_h20_hel = [
    {
        "lumi_frac": 0.675,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi_frac": 0.225,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]

# run = ilc_250_h20[0]

In [3]:
conf = {
    "parameters": {
        "g1z": 0.0,
        "ka": 0.0,
        "la": 0.0,
    },
    "obs_names": [
        "O_g1z_pos_1em05",
        "O_ka_pos_1em05",
        "O_la_pos_1em05",
    ],
    "signal_cat": "4f_sw_sl_signal",
    "signal_processes": [
        "4f_sw_sl_eLpL_signal",
        "4f_sw_sl_eLpR_signal",
        "4f_sw_sl_eRpL_signal",
        "4f_sw_sl_eRpR_signal",
    ],
    "backgrounds": [
        "4f_sl_bkg",
        "4f_not_sl",
        "2f",
        "3f",
        "5f",
        "6f",
        "higgs",
    ],
    "n_bins": 65,
}
input_path = "data/histograms/full/raw_histograms.root"

In [4]:
fit_handler = FitHandler(input_path, conf)

In [5]:
# ws = fit_handler.build_model([run])
# run_conf = (default_lumi, [default_hel])
run_conf = (ilc_250_h20_lumi, ilc_250_h20_hel)
ws = fit_handler.build_model(*run_conf)
# coupling_pars = fit_handler.coupling_pars
coupling_pars = [ws.var(name) for name in conf["parameters"]]
# obs_pars = [ws.var(f"{name}_0") for name in conf["obs_names"]]
obs_pars = [ws.var(f"{name}_{idx}") for idx in range(len(run_conf[1])) for name in conf["obs_names"] ]

{'lumi_frac': 0.675, 'e_pol': -0.8, 'p_pol': 0.3}
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
hello
{'lumi_frac': 0.225, 'e_pol': 0.8, 'p_pol': -0.3}
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la
la_la
g1z
ka
la
g1z_g1z
g1z_ka
g1z_la
ka_ka
ka_la


In [6]:
ws.Print("t")


RooWorkspace(ws) ws contents

variables
---------
(O_g1z_pos_1em05_0,O_g1z_pos_1em05_1,O_g1z_pos_1em05_2,O_g1z_pos_1em05_3,O_ka_pos_1em05_0,O_ka_pos_1em05_1,O_ka_pos_1em05_2,O_ka_pos_1em05_3,O_la_pos_1em05_0,O_la_pos_1em05_1,O_la_pos_1em05_2,O_la_pos_1em05_3,e_pol_0,e_pol_1,e_pol_2,e_pol_3,g1z,ka,la,mu_signal,p_pol_0,p_pol_1,p_pol_2,p_pol_3,runs,total_lumi)

p.d.f.s
-------
RooSimultaneous::sim_pdf[ indexCat=runs run_0=multi_gauss_0 run_1=multi_gauss_1 run_2=multi_gauss_2 run_3=multi_gauss_3 ] = 1
  RooMultiVarGaussian::multi_gauss_0[ x=(O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0) mu=(expectation_O_g1z_pos_1em05_0,expectation_O_ka_pos_1em05_0,expectation_O_la_pos_1em05_0) ] = 1
    RooFunctorBinding::expectation_O_g1z_pos_1em05_0[ function=0x2198bf80 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -209271
      RooProduct::lumi_0[ 0.675 * total_lumi ] = 1350
    RooFunctorBinding::expectation_O_ka_pos_1em05_0[ function=0x2212c300 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,m

In [7]:
# model = ws.pdf("multi_gauss_0")
model = ws.pdf("sim_pdf")

In [8]:
model.canBeExtended()
print([o.GetName() for o in obs_pars])

['O_g1z_pos_1em05_0', 'O_ka_pos_1em05_0', 'O_la_pos_1em05_0', 'O_g1z_pos_1em05_1', 'O_ka_pos_1em05_1', 'O_la_pos_1em05_1', 'O_g1z_pos_1em05_2', 'O_ka_pos_1em05_2', 'O_la_pos_1em05_2', 'O_g1z_pos_1em05_3', 'O_ka_pos_1em05_3', 'O_la_pos_1em05_3']


In [9]:
ds = ROOT.RooStats.AsymptoticCalculator.GenerateAsimovData(model, obs_pars)
ds.Print("v")

DataStore asimovDataFullModel (CountingAsimovData3)
  Contains 4 entries
  Observables: 
    1)  O_g1z_pos_1em05_0 = -209271  L(-213939 - -204604)  "O_g1z_pos_1em05_0"
    2)   O_ka_pos_1em05_0 = -438202  L(-442123 - -434281)  "O_ka_pos_1em05_0"
    3)   O_la_pos_1em05_0 = -231556  L(-237837 - -225275)  "O_la_pos_1em05_0"
    4)  O_g1z_pos_1em05_1 = -4663.31  L(-5451.11 - -3875.5)  "O_g1z_pos_1em05_1"
    5)   O_ka_pos_1em05_1 = -15888.2  L(-16699.5 - -15076.9)  "O_ka_pos_1em05_1"
    6)   O_la_pos_1em05_1 = -8909.03  L(-9977.52 - -7840.55)  "O_la_pos_1em05_1"
    7)  O_g1z_pos_1em05_2 = -8160.26  L(-9097.48 - -7223.05)  "O_g1z_pos_1em05_2"
    8)   O_ka_pos_1em05_2 = -21299.5  L(-22116.2 - -20482.8)  "O_ka_pos_1em05_2"
    9)   O_la_pos_1em05_2 = -10057.2  L(-11322.1 - -8792.38)  "O_la_pos_1em05_2"
   10)  O_g1z_pos_1em05_3 = -1566.89  L(-2022.41 - -1111.38)  "O_g1z_pos_1em05_3"
   11)   O_ka_pos_1em05_3 = -8061.11  L(-8524.31 - -7597.91)  "O_ka_pos_1em05_3"
   12)   O_la_pos_1em05_3 

In [10]:
toy_data = fit_handler.make_toy_obs_mult(ilc_250_h20_lumi, ilc_250_h20_hel)
print(toy_data)
toy_data = fit_handler.make_toy_obs_mult(ilc_250_h20_lumi, ilc_250_h20_hel, parameter_set={"g1z": 0.001, "ka": 0., "la": 0.})
print(toy_data)

[[-209628.96454317894, -437774.29268074036, -229088.14151385502], [-4512.886556540327, -15552.797415210198, -8996.942641946152], [-8127.80318375827, -20688.51306658026, -10021.498199824433], [-1515.3354927551231, -7854.470969646526, -3333.8460908676893]]
[[-211331.0356960001, -437856.76851864054, -227910.98116304353], [-4564.249542567695, -15519.127984066761, -8972.252170233553], [-8188.953400174445, -20683.25028067477, -9983.391598275373], [-1530.0219608972411, -7848.499135602722, -3326.5152868806363]]


In [11]:
fit_res = model.fitTo(ds, Save=True)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(sim_pdf) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 12.1957 ms
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_sim_pdf_asimovDataFullModel) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 2000 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 88.1126099651550305
Edm   = 2.91008356279150779e-26
Nfcn  = 51
g1z	  = 0	 +/-  0.00162991	(limited)
ka	  = 0	 +/-  0.00665126	(limited)
la	  = 0	 +/-  0.00269419	(limited)
total_lumi	  = 2000	 +/-  25.8511	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 582.634 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       88.11260997 Edm =   3.216655602e-26 NCalls =     17
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 88.11260997
  Edm           : 3.216655602e-26
  Internal parameters:	[                0                0                0                0]	
  Internal gradient  :	[ -7.057509662e-11 -3.528754831e-11  1.411501932e-10                0]	
  Internal covariance matrix:
[[  4.2126126e-06              0              0              0]
 [              0  1.8029481e-05              0              0]
 [              0              0  4.2780573e-06              0]
 [              0              0              0   0.0014459177]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 2000
Info in <Minuit2>: Variabl

In [12]:
fit_res.covarianceMatrix().Print()


4x4 matrix is as follows

     |      0    |      1    |      2    |      3    |
---------------------------------------------------------
   0 |  2.657e-06   7.125e-06   3.771e-06    -0.03112 
   1 |  7.125e-06   4.424e-05   1.594e-05      -0.167 
   2 |  3.771e-06   1.594e-05   7.259e-06    -0.06433 
   3 |   -0.03112      -0.167    -0.06433         672 



In [13]:
nll = model.createNLL(ds, EvalBackend="cpu")

[#1] INFO:Fitting -- RooAbsPdf::fitTo(sim_pdf) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- Creation of NLL object took 1.21883 ms


In [14]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_sim_pdf_asimovDataFullModel) Summation contains a RooNLLVar, using its error level


In [15]:
%time
nll_minimizer.migrad()

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 6.68 μs


0

Minuit2Minimizer: Minimize with max-calls 2000 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 88.1126099651550305
Edm   = 0
Nfcn  = 40
g1z	  = 0	 +/-  0.00161632	(limited)
ka	  = 0	 +/-  0.00657772	(limited)
la	  = 0	 +/-  0.00266321	(limited)
total_lumi	  = 2000	 +/-  25.5475	(limited)


Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 500.706 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       88.11260997 Edm =                 0 NCalls =     17
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 88.11260997
  Edm           : 0
  Internal parameters:	[                0                0                0                0]	
  Internal gradient  :	[                0                0                0                0]	
  Internal covariance matrix:
[[  4.2126125e-06              0              0              0]
 [              0   1.802948e-05              0              0]
 [              0              0  4.2780572e-06              0]
 [              0              0              0   0.0014459177]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 2000
Info in <Minuit2>: VariableMetricBuilder

In [16]:
pll0 = nll.createProfile({coupling_pars[0]})
pll1 = nll.createProfile({coupling_pars[1]})
pll2 = nll.createProfile({coupling_pars[2]})

In [17]:
frame0 = coupling_pars[0].frame(Range=(-0.004, 0.004), Title=f";{list(conf["parameters"])[0]};-#Delta NLL")
frame1 = coupling_pars[1].frame(Range=(-0.004, 0.004), Title=f";{list(conf["parameters"])[1]};-#Delta NLL")
frame2 = coupling_pars[2].frame(Range=(-0.004, 0.004), Title=f";{list(conf["parameters"])[2]};-#Delta NLL")
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

In [18]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_sim_pdf_asimovDataFullModel) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) minimum found at (g1z=0)
..........................................................................................................................................................................................................
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[ka]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_sim_pdf_asimovDataFullModel) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- Ro

In [19]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(2)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")
c0.SaveAs("plots/fit/ll_pll_g1z.pdf")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(2)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")
c1.SaveAs("plots/fit/ll_pll_ka.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(2)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")
c2.SaveAs("plots/fit/ll_pll_la.pdf")

Info in <TCanvas::Print>: pdf file plots/fit/ll_pll_g1z.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fit/ll_pll_ka.pdf has been created
Info in <TCanvas::Print>: pdf file plots/fit/ll_pll_la.pdf has been created
